<a href="https://colab.research.google.com/github/lbralexis/OrangeGIt/blob/main/Git_Orange_VF2_Stable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MODE OPERATOIRE**
- Ouvrir le code du bench Orange Google Collab via raccourci
- Cliquer sur la flèche pour importer les librairies
- Insérer le fichier des urls (dossier M:\Projet\DTX\PVMC\Bench\Bench auto)
- Lancer le code en appuyant sur la flèche
- Aller en bas et choisir si on veut l'exhaustivité ou le 1er onglet
- Télécharger le fichier csv, puis aller le coller dans l'onglet DATA ORANGE du bench

In [1]:
# install chromium, its driver, and selenium
!apt update
!apt install libu2f-udev libvulkan1
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!wget https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip
!unzip -j chromedriver-linux64.zip chromedriver-linux64/chromedriver -d /usr/local/bin/
!pip install selenium chromedriver_autoinstaller
!pip install pytz

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [973 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,258 kB]
Get:13 http://archive.ubuntu.com

In [5]:
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import time
import pandas as pd
from datetime import datetime
import pytz

# Install chromedriver
chromedriver_autoinstaller.install()

# Initialize an empty list to store data
data = []

# Configure headless Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize WebDriver
wd = webdriver.Chrome(options=options)

urls = ['https://boutique.orange.fr/mobile/details/apple-iphone-11-noir-64go-skp?offer=forfait-200go-5g']


# Traiter chaque URL
for url in urls:
    print(f"Processing URL: {url}")
    try:
        # Placez ici le code pour traiter chaque URL
        pass
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# Define a function to extract offer from URL
def extract_offer(url):
    match = re.search(r'offer=([^&]+)', url)
    if match:
        return match.group(1)
    else:
        return "N/A"

# Process each URL
for url in urls:
    print(f"Processing URL: {url}")
    try:
        # Open the target URL
        wd.get(url)

        # Wait for page load (adjust as needed)
        time.sleep(2)

        # Extract and process data as before
        h1_element = wd.find_element(By.CSS_SELECTOR, "#mobile-details > div > div.bg-gray2 > div > div.d-flex.flex-column.flex-md-row.mt-1 > h1")
        h1_text = h1_element.text
        print(f"Found model: {h1_text}")

        # Extract capacity elements
        capacity_elements = wd.find_elements(By.CSS_SELECTOR, "#mobile-details > div > div.bg-gray2 > div > div:nth-child(5) > div.mb-2.w-100 > div.bg-white.px-1.px-md-2.py-1 > fieldset:nth-child(3) > span")
        print(f"Found {len(capacity_elements)} capacities")

        for element in capacity_elements:
            try:
                capacity = element.text
                print(f"Processing capacity: {capacity}")
                element.click()

                # Wait for price element to be present
                price_amount_element = WebDriverWait(wd, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#mobile-details > div > div.bg-gray2 > div > div:nth-child(5) > div.mb-2.w-100 > div.bg-white.px-1.px-md-2.py-1 > div > div.d-lg-flex.justify-content-between > div > div.ob1-price.d-block > p > span.ob1-price-amount"))
                )
                price_amount_text = price_amount_element.text
                print(f"Found promo price: {price_amount_text}")

                try:
                    # Extract data element
                    data_element = wd.find_element(By.CSS_SELECTOR, "#mobile-details > div > div.bg-gray2 > div > div:nth-child(5) > div.mb-2.w-100 > div.bg-white.px-1.px-md-2.pb-1 > div:nth-child(2) > div")
                    data_text = data_element.text
                    print(f"Found data: {data_text}")

                    # Use regular expression to extract date
                    date_pattern = r'\d{2}/\d{2}/\d{4}'  # Assuming date format is dd/mm/yyyy
                    match = re.search(date_pattern, data_text)
                    if match:
                        extracted_date = match.group()
                        print(f"Extracted date: {extracted_date}")
                    else:
                        extracted_date = "N/A"
                except NoSuchElementException:
                    extracted_date = "N/A"  # or any default value you prefer

                # Replace "N/A" with an empty string
                if extracted_date == "N/A":
                    extracted_date = ""

                # Append data to list
                data.append({
                    'Forfait': extract_offer(url),  # Extract offer from URL
                    'Modèle': h1_text,
                    'Capacité': capacity,
                    'Prix Comptant': int(re.search(r'\d+', price_amount_text).group(0)),
                    'Date': extracted_date
                })

            except Exception as e:
                print(f"Error extracting data for capacity {element.text}:", e)

    except Exception as e:
        print(f"Error processing URL {url}:", e)

# Close the WebDriver
wd.quit()

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Add the 'Operator' column with value 'Orange'
df.insert(0, 'Operator', 'Orange')

# Define the timezone for Paris
paris_tz = pytz.timezone('Europe/Paris')

# Get the current UTC time and convert it to Paris timezone
utc_time = datetime.utcnow().replace(tzinfo=pytz.utc)
paris_time = utc_time.astimezone(paris_tz)

# Generate the filename with the current date and time
date_str = paris_time.strftime("Orange-Bench-%Y-%m-%d_%Hh%M-%S.csv")

# Export the DataFrame to a CSV file
df.to_csv(date_str, index=False)



Processing URL: https://boutique.orange.fr/mobile/details/apple-iphone-11-noir-64go-skp?offer=forfait-200go-5g
Processing URL: https://boutique.orange.fr/mobile/details/apple-iphone-11-noir-64go-skp?offer=forfait-200go-5g
Found model: Apple iPhone 11
Found 3 capacities
Processing capacity: 64 Go
Found promo price: 219 €
Found data: 20 € de remise immédiate
jusqu'au 10/07/2024
En savoir plus
Extracted date: 10/07/2024
Processing capacity: 128 Go
Found promo price: 269 €
Found data: 20 € de remise immédiate
jusqu'au 10/07/2024
En savoir plus
Extracted date: 10/07/2024
Processing capacity: 256 Go
Found promo price: 269 €
Found data: 20 € de remise immédiate
jusqu'au 10/07/2024
En savoir plus
Extracted date: 10/07/2024
